In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import os
import requests

INPUT_SIZE = 10
HIDDEN_SIZE = 4
CSV_FILENAME = 'mitbih_test.csv'

# ✅ CORRECTED LINK: Points to the RAW data on your GitHub
# I converted 'github.com' -> 'raw.githubusercontent.com' and removed 'blob'
DATA_URL = "https://raw.githubusercontent.com/bshihab/pico_hrm_integrity/main/hospital_sim/mitbih_test.csv"

def force_download_data():
    """
    Downloads the file from your specific GitHub repo.
    """
    # Clean up old empty files if they exist
    if os.path.exists(CSV_FILENAME):
        if os.path.getsize(CSV_FILENAME) < 1000:
            os.remove(CSV_FILENAME)
        else:
            print(f"✅ File already exists ({os.path.getsize(CSV_FILENAME)/1024:.1f} KB).")
            return

    print(f"⬇️ Downloading from your GitHub...")
    try:
        response = requests.get(DATA_URL)
        if response.status_code == 200:
            with open(CSV_FILENAME, 'wb') as f:
                f.write(response.content)
            print("✅ Download successful!")
        else:
            print(f"❌ Download failed (Status: {response.status_code}).")
            print("NOTE: If this failed, is your GitHub repo Private? It must be Public for this to work.")
            return
    except Exception as e:
        print(f"❌ Network error: {e}")
        return

def load_data():
    force_download_data()

    if not os.path.exists(CSV_FILENAME):
        print("❌ Error: File not found.")
        return None, None

    print("Loading dataset...")
    try:
        df = pd.read_csv(CSV_FILENAME, header=None)
    except Exception as e:
        print(f"❌ CSV Error: {e}")
        return None, None
        
    # distinguish Normal (0.0) vs PVC (2.0)
    df = df[df[187].isin([0.0, 2.0])]
    
    # Extract the Peak of the heartbeat (samples 20 to 30)
    X = df.iloc[:, 20:20+INPUT_SIZE].values
    
    # Labels: 0.0 -> 0 (Safe), 2.0 -> 1 (Danger)
    y = df.iloc[:, 187].apply(lambda x: 1 if x == 2.0 else 0).values
    
    return X, y

def export_to_c(model):
    print("\n\n" + "="*40)
    print("PASTE THIS INTO firmware/src/main.c")
    print("="*40 + "\n")
    
    # Layer 1 (Hidden)
    weights1, biases1 = model.layers[0].get_weights()
    print(f"// Layer 1 Weights ({INPUT_SIZE}x{HIDDEN_SIZE})")
    print(f"const float W1[{INPUT_SIZE}][{HIDDEN_SIZE}] = {{")
    for i in range(INPUT_SIZE):
        row = ", ".join([f"{w:.4f}" for w in weights1[i]])
        print(f"    {{{row}}},")
    print("};")
    
    print(f"\n// Layer 1 Biases ({HIDDEN_SIZE})")
    row = ", ".join([f"{b:.4f}" for b in biases1])
    print(f"const float B1[{HIDDEN_SIZE}] = {{{row}}};")

    # Layer 2 (Output)
    weights2, biases2 = model.layers[1].get_weights()
    print(f"\n// Layer 2 Weights ({HIDDEN_SIZE})")
    row = ", ".join([f"{w[0]:.4f}" for w in weights2])
    print(f"const float W2[{HIDDEN_SIZE}] = {{{row}}};")
    
    print(f"\n// Layer 2 Bias (Scalar)")
    print(f"const float B2 = {biases2[0]:.4f};")
    print("\n" + "="*40)

def main():
    X, y = load_data()
    if X is None: return

    print(f"Training on {len(X)} heartbeats...")

    model = Sequential([
        Dense(HIDDEN_SIZE, input_dim=INPUT_SIZE, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)
    
    export_to_c(model)

if __name__ == "__main__":
    main()

⬇️ Downloading from your GitHub...
✅ Download successful!
Loading dataset...
Training on 19566 heartbeats...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9242 - loss: 0.4273
Epoch 2/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9264 - loss: 0.2884
Epoch 3/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9270 - loss: 0.2525
Epoch 4/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9260 - loss: 0.2487
Epoch 5/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9259 - loss: 0.2462
Epoch 6/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9252 - loss: 0.2439
Epoch 7/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9271 - loss: 0.2390
Epoch 8/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9263 - loss: 0.2377
Epoch 9/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9251 - loss: 0.2426
Epoch 10/10
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9282 - loss: 0.2328


PASTE THIS INTO firmware/src/main.c

// Layer 1 Weights (10x4)
const float W1[10][4] = {
    {-0.4644, 0.4617, 0.5796, -0.6543},
    {0.5